In [18]:
import uuid
import os
import asyncio
from IPython.display import Image
from IPython.core.display import HTML
import logging

import dotenv
import instagrapi
import httpx
import aiofiles

from config import InstaSettings, BotSettings
from services.insta_service import InstaService
from dto.instagram import *
from utils.custom_telegram_client import MegaTelegramClient

dotenv.load_dotenv()

True

In [2]:
insta_settings = InstaSettings()
client = instagrapi.Client()
insta_service = InstaService(logger=logging.getLogger('jupyter'), config=insta_settings, insta_client=client)

In [3]:
client.login(username=insta_settings.USERNAME, password=insta_settings.PASSWORD)

True

In [4]:
urls = [
    'https://www.instagram.com/p/DKz6hiItQ_R/',  # picture
    'https://www.instagram.com/reel/DIlaS8VuzT2/',  # reel
    'https://www.instagram.com/p/DKcL_DONecG/',  # picture album
    'https://www.instagram.com/p/DKXAZ2CtnI3/',  # video album
]

In [5]:
media_info = [
    await insta_service.get_media_info_by_link(url)
    for url in urls
]

In [7]:
MY_ID = 336619540
bot_settings = BotSettings()
bot = MegaTelegramClient(
    session=bot_settings.APP_NAME,
    api_id=bot_settings.API_ID,
    api_hash=bot_settings.API_HASH,
    system_version=f"4.16.30-vx{bot_settings.APP_NAME}",
    loop=asyncio.get_running_loop(),
    di_container=None,
    logger=logging.getLogger('jupyter'),
)

In [22]:
# image
httpx_client = httpx.AsyncClient()

async def save(tempdir: str, url: str, ext: str):
    path = os.path.join(tempdir, f"{uuid.uuid4().hex}.{ext}")
    response = await httpx_client.get(url)
    async with aiofiles.open(path, 'wb') as f:
        async for chunk in response.aiter_bytes():
            await f.write(chunk)
        return path


async with bot, aiofiles.tempfile.TemporaryDirectory() as tmp_dir:
    file_name = await save(tmp_dir, str(media_info[0].thumbnail_url), 'jpg')
    await bot.send_message(
        entity=MY_ID,
        file=file_name,
        message=urls[0],
        silent=True,
    )
    Image(filename=file_name)

In [11]:
# reel
async with bot:
    await bot.send_message(
        entity=MY_ID,
        file=str(media_info[1].video_url),
        message=urls[1],
        silent=True,
    )